In [184]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
from IPython import display

import numpy as np
import pandas as pd
from scipy.sparse.linalg import svds
from numpy.linalg import norm, svd

import itertools
import pickle
import math
import re

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from gensim.models import word2vec, Word2Vec
import logging

from word2vec_as_MF import Word2vecMF
from functions import *

import time
dimension = 20
model =  Word2vecMF()
dictionary = {'Evaluation':[],'BFGD':[],'RO':[], 'word2vec':[], 'number of pairs':[]}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [185]:
model.load_matrices(from_file='enwik-200/matrices2.npz');
sentences=parse(from_file='data/x2')
skip = Word2Vec(sentences, size=dimension, min_count=5)

Parsing sentences from training set


In [186]:
Path = 'datasets/men3000.csv'
print(Path)

df = pd.read_csv(Path,header=None, delimiter=';');
score1, _, _, chosen_pairs = corr_experiment(model = model, benchmark = df, 
                                            from_folder='enwik-200/AMiter_fromPMM_D2_dim20_step1e-05_0.006',
                                           ITER=[190000]);
print(score1, len(chosen_pairs))

score2, _, _, chosen_pairs = corr_experiment(model = model, benchmark = df,
                                            from_folder='enwik-200/PSiter_fromSVD_dim20_step5e-05_0.0',
                                           ITER=[360000]);
print(score2, len(chosen_pairs))

score3, _, _, chosen_pairs = corr_word2vec(skip ,df, model_vocab=model.vocab)
print(score3, len(chosen_pairs))

dictionary['Evaluation'].append(Path.split('/')[1].split('.')[0])
dictionary['BFGD'].append(score1[0])
dictionary['RO'].append(score2[0])
dictionary['word2vec'].append(score3)
dictionary['number of pairs'].append(len(chosen_pairs))

datasets/men3000.csv
[0.20834729] 60
[0.05638974] 60
-0.14442223771888119 60


In [187]:
Path = 'datasets/verb143.csv'
print(Path)
model.load_matrices(from_file='enwik-200/matrices2.npz');
df = pd.read_csv(Path,header=None, delimiter=';');
score1, _, _, chosen_pairs = corr_experiment(model = model, benchmark = df, 
                                            from_folder='enwik-200/AMiter_fromPMM_D2_dim20_step1e-05_0.006',
                                           ITER=[190000]);
print(score1, len(chosen_pairs))

score2, _, _, chosen_pairs = corr_experiment(model = model, benchmark = df,
                                            from_folder='enwik-200/PSiter_fromSVD_dim20_step5e-05_0.0',
                                           ITER=[360000]);
print(score2, len(chosen_pairs))

score3, _, _, chosen_pairs = corr_word2vec(skip ,df, model_vocab=model.vocab)
print(score3, len(chosen_pairs))

dictionary['Evaluation'].append(Path.split('/')[1].split('.')[0])
dictionary['BFGD'].append(score1[0])
dictionary['RO'].append(score2[0])
dictionary['word2vec'].append(score3)
dictionary['number of pairs'].append(len(chosen_pairs))

datasets/verb143.csv
[0.0227791] 11
[-0.21412356] 11
-0.1138955118945513 11


In [188]:
Path = 'datasets/wordsim353.csv'
print(Path)
model.load_matrices(from_file='enwik-200/matrices2.npz');
df = pd.read_csv(Path,header=None, delimiter=';');
score1, _, _, chosen_pairs = corr_experiment(model = model, benchmark = df, 
                                            from_folder='enwik-200/AMiter_fromPMM_D2_dim20_step1e-05_0.006',
                                           ITER=[190000]);
print(score1, len(chosen_pairs))

score2, _, _, chosen_pairs = corr_experiment(model = model, benchmark = df,
                                            from_folder='enwik-200/PSiter_fromSVD_dim20_step5e-05_0.0',
                                           ITER=[360000]);
print(score2, len(chosen_pairs))

score3, _, _, chosen_pairs = corr_word2vec(skip ,df, model_vocab=model.vocab)
print(score3, len(chosen_pairs))

dictionary['Evaluation'].append(Path.split('/')[1].split('.')[0])
dictionary['BFGD'].append(score1[0])
dictionary['RO'].append(score2[0])
dictionary['word2vec'].append(score3)
dictionary['number of pairs'].append(len(chosen_pairs))

datasets/wordsim353.csv
[0.25295573] 32
[-0.17102007] 32
0.3609201738190157 32


In [189]:
Path = 'datasets/MTURK-771.csv'
print(Path)
model.load_matrices(from_file='enwik-200/matrices2.npz');
df = pd.read_csv(Path,header=None, delimiter=',');
score1, _, _, chosen_pairs = corr_experiment(model = model, benchmark = df, 
                                            from_folder='enwik-200/AMiter_fromPMM_D2_dim20_step1e-05_0.006',
                                           ITER=[190000]);
print(score1, len(chosen_pairs))

score2, _, _, chosen_pairs = corr_experiment(model = model, benchmark = df,
                                            from_folder='enwik-200/PSiter_fromSVD_dim20_step5e-05_0.0',
                                           ITER=[360000]);
print(score2, len(chosen_pairs))

score3, _, _, chosen_pairs = corr_word2vec(skip ,df, model_vocab=model.vocab)
print(score3, len(chosen_pairs))

dictionary['Evaluation'].append(Path.split('/')[1].split('.')[0])
dictionary['BFGD'].append(score1[0])
dictionary['RO'].append(score2[0])
dictionary['word2vec'].append(score3)
dictionary['number of pairs'].append(len(chosen_pairs))

datasets/MTURK-771.csv
[0.40262635] 30
[0.32472739] 30
0.4751836660409047 30


In [190]:
Path = 'datasets/mturk287.csv'
print(Path)
model.load_matrices(from_file='enwik-200/matrices2.npz');
df = pd.read_csv(Path,header=None, delimiter=';');
score1, _, _, chosen_pairs = corr_experiment(model = model, benchmark = df, 
                                            from_folder='enwik-200/AMiter_fromPMM_D2_dim20_step1e-05_0.006',
                                           ITER=[190000]);
print(score1, len(chosen_pairs))

score2, _, _, chosen_pairs = corr_experiment(model = model, benchmark = df,
                                            from_folder='enwik-200/PSiter_fromSVD_dim20_step5e-05_0.0',
                                           ITER=[360000]);
score3, _, _, chosen_pairs = corr_word2vec(skip ,df, model_vocab=model.vocab)
print(score3, len(chosen_pairs))

dictionary['Evaluation'].append(Path.split('/')[1].split('.')[0])
dictionary['BFGD'].append(score1[0])
dictionary['RO'].append(score2[0])
dictionary['word2vec'].append(score3)
dictionary['number of pairs'].append(len(chosen_pairs))

datasets/mturk287.csv
[0.36969697] 10
0.6969696969696969 10


In [191]:
Path = 'datasets/rw2034.csv'
print(Path)
model.load_matrices(from_file='enwik-200/matrices2.npz');
df = pd.read_csv(Path,header=None, delimiter=';');
score1, _, _, chosen_pairs = corr_experiment(model = model, benchmark = df, 
                                            from_folder='enwik-200/AMiter_fromPMM_D2_dim20_step1e-05_0.006',
                                           ITER=[190000]);
print(score1, len(chosen_pairs))

score2, _, _, chosen_pairs = corr_experiment(model = model, benchmark = df,
                                            from_folder='enwik-200/PSiter_fromSVD_dim20_step5e-05_0.0',
                                           ITER=[360000]);
print(score2, len(chosen_pairs))

score3, _, _, chosen_pairs = corr_word2vec(skip ,df, model_vocab=model.vocab)
print(score3, len(chosen_pairs))

dictionary['Evaluation'].append(Path.split('/')[1].split('.')[0])
dictionary['BFGD'].append(score1[0])
dictionary['RO'].append(score2[0])
dictionary['word2vec'].append(score3)
dictionary['number of pairs'].append(len(chosen_pairs))

datasets/rw2034.csv
[1.] 3
[0.5] 3
0.5 3


In [192]:
Path = 'datasets/simlex999.csv'
print(Path)
model.load_matrices(from_file='enwik-200/matrices2.npz');
df = pd.read_csv(Path,header=None, delimiter=';');
score1, _, _, chosen_pairs = corr_experiment(model = model, benchmark = df, 
                                            from_folder='enwik-200/AMiter_fromPMM_D2_dim20_step1e-05_0.006',
                                           ITER=[190000]);
print(score1, len(chosen_pairs))

score2, _, _, chosen_pairs = corr_experiment(model = model, benchmark = df,
                                            from_folder='enwik-200/PSiter_fromSVD_dim20_step5e-05_0.0',
                                           ITER=[360000]);
print(score2, len(chosen_pairs))

score3, _, _, chosen_pairs = corr_word2vec(skip ,df, model_vocab=model.vocab)
print(score3, len(chosen_pairs))

dictionary['Evaluation'].append(Path.split('/')[1].split('.')[0])
dictionary['BFGD'].append(score1[0])
dictionary['RO'].append(score2[0])
dictionary['word2vec'].append(score3)
dictionary['number of pairs'].append(len(chosen_pairs))

datasets/simlex999.csv
[-0.04854133] 42
[-0.20640195] 42
-0.062236629053625904 42


In [193]:
df = pd.DataFrame(dictionary)
cols = df.columns.tolist()
j = cols.index('Evaluation')
cols[0],cols[j]=cols[j], cols[0]
j = cols.index('number of pairs')
cols[-1],cols[j]=cols[j], cols[-1]
df=df[cols]
df.style


,Evaluation,BFGD,RO,word2vec,number of pairs
0,men3000,0.208347,0.0563897,-0.144422,60
1,verb143,0.0227791,-0.214124,-0.113896,11
2,wordsim353,0.252956,-0.17102,0.36092,32
3,MTURK-771,0.402626,0.324727,0.475184,30
4,mturk287,0.369697,0.490909,0.69697,10
5,rw2034,1,0.5,0.5,3
6,simlex999,-0.0485413,-0.206402,-0.0622366,42
